# DATA PREPROCESSING AND VISUALIZATION 
**Mục tiêu:**
1. **Data Preprocessing:** Làm sạch toàn bộ dữ liệu thô (`Missing value`,`Feature Engineering`,`Tranform`,`Encoding`,`Label`) , lưu cleaned_data vào thư mục processed_data
2. **Visualization:** Trực quan hóa dựa vào các câu hỏi ý nghĩa
3. **Insights and Conclusion:** Rút ra được góc nhìn hay và kết luận 

## 1.Tải dữ liệu thô (raw_data) và định dạng các cột
**Mục tiêu:** Tải dữ liệu thô và loại bỏ các cột không cần thiết , định dạng các kiểu dữ liệu đúng cho từng cột 

### Import các thư viện cần thiết và src

In [2]:
import sys 
import os
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
import numpy as np 
import seaborn as sns
import data_processing as dp
import visualization as vi
sns.set_theme(style="whitegrid", palette="pastel")

### Tải dữ liệu thô và đinh dạng

In [3]:
file_path = '../data/raw/BankChurners.csv'
data, header = dp.load_data(file_path)
col_to_index = {col_name: i for i ,col_name in enumerate(header)} #Trích xuất chỉ số các header
#Lấy tên các cột không cần thiêt
cols_to_drop = ["CLIENTNUM",
                "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
               "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"]

#Láy tên các cột số
numerical_cols = ['Customer_Age', 'Dependent_count', 'Months_on_book', 
    'Total_Relationship_Count', 'Months_Inactive_12_mon', 
    'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 
    'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Amt_Chng_Q4_Q1',
    'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']
#Lấy tên các cột categorical
categorical_cols = ['Attrition_Flag', 'Gender', 'Education_Level', 
    'Marital_Status', 'Income_Category', 'Card_Category']
#Tạo một dictionary để lưu trữ các dữ liệu theo đúng kiểu dữ liệu của nó , và đã loại bỏ các cột không cần thiết
clean_data_store = {}
for col_name in header:
    if col_name in cols_to_drop:
        continue
    col_str = data[:,col_to_index[col_name]] #Trích xuất data của cột đó
    if col_name in numerical_cols:
        col_float, null_mask = dp.get_valid_numerical_array(col_str)
        col_float[null_mask] = np.nan
        clean_data_store[col_name] = col_float 
    elif col_name in categorical_cols:
        clean_data_store[col_name] = col_str

for key , values in clean_data_store.items():
    print(key,values)

Attrition_Flag ['Existing Customer' 'Existing Customer' 'Existing Customer' ...
 'Attrited Customer' 'Attrited Customer' 'Attrited Customer']
Customer_Age [45. 49. 51. ... 44. 30. 43.]
Gender ['M' 'F' 'M' ... 'F' 'M' 'F']
Dependent_count [3. 5. 3. ... 1. 2. 2.]
Education_Level ['High School' 'Graduate' 'Graduate' ... 'High School' 'Graduate'
 'Graduate']
Marital_Status ['Married' 'Single' 'Married' ... 'Married' 'Unknown' 'Married']
Income_Category ['$60K - $80K' 'Less than $40K' '$80K - $120K' ... 'Less than $40K'
 '$40K - $60K' 'Less than $40K']
Card_Category ['Blue' 'Blue' 'Blue' ... 'Blue' 'Blue' 'Silver']
Months_on_book [39. 44. 36. ... 36. 36. 25.]
Total_Relationship_Count [5. 6. 4. ... 5. 4. 6.]
Months_Inactive_12_mon [1. 1. 1. ... 3. 3. 2.]
Contacts_Count_12_mon [3. 2. 0. ... 4. 3. 4.]
Credit_Limit [12691.  8256.  3418. ...  5409.  5281. 10388.]
Total_Revolving_Bal [ 777.  864.    0. ...    0.    0. 1961.]
Total_Amt_Chng_Q4_Q1 [1.335 1.541 2.594 ... 0.819 0.535 0.703]
Total_Tra